<a href="https://colab.research.google.com/github/vitroid/ClustIce/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
try:
    import google.colab
    ! pip install py3Dmol genice-core genice2 git+https://github.com/vitroid/clustice.git
except:
    pass

In [1]:
from logging import getLogger, INFO, DEBUG
import numpy as np
import networkx as nx
import py3Dmol

import genice_core
from clustice.serialize import serialize
from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.water import tip4p
from clustice import graph
from cycless import cycles

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = graph.great_icosahedron(12, separation=L)
g = graph.great_decahedron(4)
# g = graph.small_barrelan()
# g = graph.large_barrelan()
# g = graph.twistane()
# g = graph.adamantane()
# g = nx.cycle_graph(6) # hexagon
# g = nx.cycle_graph(7) # heptagon
# g = nx.cubical_graph() # cubic octamer
# g = nx.dodecahedral_graph()

# count cycles in the graph
cyclehist = {}
for cycle in cycles.cycles_iter(g, maxsize=5):
    nnode = len(cycle)
    cyclehist[nnode] = cyclehist.get(nnode, 0) + 1
print(cyclehist)


if "pos" in g.nodes[0]:
    # extract the embedded coords in g
    mol_positions = np.array([g.nodes[v]["pos"] for v in g])
else:
    # estimate of the positions of the nodes
    mol_positions = make_layout(g, edgelen=L)

# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = genice_core.ice_graph(
    g, vertexPositions=mol_positions, dipoleOptimizationCycles=1000
)
# dg = ice_graph(g)

# get the unique id for the graph
# id = serialize(dg)
# print(id)

# put water molecules
gro = render(
    dg,
    mol_positions,
    water_model=tip4p,
    cell_matrix=np.array([[10.0, 0.0, 0.0], [0.0, 10.0, 0.0], [0.0, 0.0, 10.0]]),
    shift=np.array([0.5, 0.5, 0.5]),
)
with open("save.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

{5: 9}


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Rebuild the cluster from id.


In [3]:
import networkx as nx

from clustice.geometry import make_layout
from clustice.gromacs import render
from clustice.serialize import deserialize
from clustice.water import tip4p

import py3Dmol

s = "1.2.3+6.19.3.4+6.7.14.1+7.8+13.0+9.10+12.16.12.13+15.5.15+17.4+18.11.0+18"

dg = deserialize(s)

# Make layout of the nodes
g = nx.Graph(dg)
L = 0.27
mol_positions = make_layout(g, edge_length=L)

# put water molecules
gro = render(dg, mol_positions, water_model=tip4p)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
import networkx as nx

from genice2.genice import GenIce
from genice2.plugin import Format, Lattice
from clustice.gromacs import render
from clustice.water import tip4p
import genice_core

import py3Dmol

lattice = Lattice("ice1h")
formatter = Format("raw", stage=(1, 2))
raw = GenIce(lattice, signature="Ice Ih", rep=(3, 3, 3)).generate_ice(formatter)

# graph is the topology of the hydrogen-bond network
g = nx.Graph(raw["graph"])
# reppositions contains the positions of CoM of water in fractional coordinate
mol_positions = raw["reppositions"]
# repcell is the cell matrix (transposed)
cell_matrix = raw["repcell"]

# set orientations of the hydrogen bonds.
# It automatically depolarizes by optimizing the arrangement.
dg = genice_core.ice_graph(
    g,
    vertexPositions=mol_positions,
    isPeriodicBoundary=True,
    dipoleOptimizationCycles=200,
)

# put water molecules
gro = render(
    dg,
    mol_positions @ cell_matrix,
    water_model=tip4p,
    cell_matrix=cell_matrix,
)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})
view.addUnitCell()
view.zoomTo()
view.show()

INFO:root:Cell dimension:
INFO:root:  a = 7.84813412606925
INFO:root:  b = 7.37735062301457
INFO:root:  c = 9.06573834219084
INFO:root:  A = 90.0
INFO:root:  B = 90.0
INFO:root:  C = 90.0
INFO:root:HB connectivity is not defined.
INFO:root:Bond length (specified): 3
INFO:root:Target Density: 0.92
INFO:root:Original Density: 0.0009111328713390615
INFO:root:Bond length (scaled, nm): 0.29903306849378936
INFO:root:__init__: end.
INFO:root:__init__: 5 ms
INFO:root:Stage1: Replicate water molecules to make a repeated cell.
INFO:root:  Number of water molecules: 432
INFO:root:  Pairs are not given explicitly.
INFO:root:  Estimating the bonds according to the pair distances.
INFO:root:  Number of water nodes: 16
INFO:root:Cell dimension:
INFO:root:  a = 0.7822838765564372
INFO:root:  b = 0.7353572647182051
INFO:root:  c = 0.9036518515423753
INFO:root:  A = 90.0
INFO:root:  B = 90.0
INFO:root:  C = 90.0
INFO:root:Stage1: end.
INFO:root:Stage1: 6 ms
INFO:root:Hook1: Replicate water molecules to 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Benchmark tests


In [5]:
from clustice.geometry import make_layout
from clustice.graph import great_icosahedron

from logging import getLogger, INFO

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

# note: g must be a graph whose labels start from 0.
# g = nx.dodecahedral_graph() # dodecahedral 20mer
g = great_icosahedron(1)
mol_positions = make_layout(g, edge_length=L)
print(len(mol_positions))

/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
INFO:root:1 0.7579917196546688 RG
INFO:root:2 0.647962328195014 RG
INFO:root:4 0.5363145443093116 RG
INFO:root:8 0.47185480775555055 RG
INFO:root:16 0.45751493827193473 RG
INFO:root:32 0.4569351602494919 RG
INFO:root:64 0.4569342557485291 RG


100


In [9]:
# estimate of the positions of the nodes
mol_positions = make_layout(g, edge_length=L)

/Users/matto/.local/share/virtualenvs/ClustIce-TrkXxxIw/lib/python3.11/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
INFO:root:1 0.7580608469541879 RG
INFO:root:2 0.6480040493933352 RG
INFO:root:4 0.5363388598876981 RG
INFO:root:8 0.4718600879485308 RG
INFO:root:16 0.4575151242265524 RG
INFO:root:32 0.4569351587913154 RG
INFO:root:64 0.45693425574230845 RG


In [10]:
dg = genice_core.ice_graph(g, vertexPositions=mol_positions)

In [11]:
dg = genice_core.ice_graph(g)

In [ ]:
# great_icosahedron
# size  nnode   constellation        depolarization       ice rule
benchmark = [
    [0, 20, 0.146, 0.00612, 0.000410],
    [1, 100, 0.676, 0.00865, 0.00229],
    [2, 280, 3.14, 0.0178, 0.00569],
    [3, 600, 51.4, 0.024, 0.0133],
]